In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array

### Load Data 

In [ ]:
BTC_df = pd.read_csv("/kaggle/input/high-frequency-crypto-limit-order-book-data/BTC_1min.csv")

In [ ]:
BTC_df.head()

In [ ]:
BTC_df.drop("Unnamed: 0", inplace = True, axis = 1)
BTC_df.drop("system_time", inplace = True, axis = 1)

### Split data in to train and test

In [ ]:
BTC_df.shape

In [ ]:
data_length = BTC_df.shape[0]

train_length = round(0.8*data_length)
train_length

### Standardize by mean and std in training data

In [ ]:
mean_df = BTC_df[:train_length].mean()
std_df = BTC_df[:train_length].std()

normalized_df = (BTC_df - mean_df) / std_df

### Create Sliding Windows

In [ ]:
# Set window size
window_size = 30

# Windows for training dataset
windows_train = timeseries_dataset_from_array(normalized_df[:(train_length)], normalized_df["midpoint"][window_size:(train_length+window_size)], window_size, sequence_stride=1, sampling_rate=1,
    batch_size=64, shuffle=False, seed=None, start_index=None, end_index=None
)


# Windows for test dataset
windows_test = timeseries_dataset_from_array(normalized_df[train_length:(data_length-window_size)], normalized_df["midpoint"][train_length + window_size:], 30, sequence_stride=1, sampling_rate=1,
    batch_size=64, shuffle=False, seed=None, start_index=None, end_index=None
)

### Dense Model

In [ ]:
model_dense = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[30, 154]),
    keras.layers.Dense(16),
    keras.layers.Dense(1)
    ])
    
model_dense.compile(loss="mse", optimizer="adam", metrics=["mse"])
history = model_dense.fit(windows_train, epochs=10,verbose=1)

In [ ]:
# Predict
preds_dense = model_dense.predict(windows_test)

# Remove standardization from predictions

preds_dense_unstd = preds_dense * std_df["midpoint"] + mean_df["midpoint"]


# Plot 
plt.plot(preds_dense_unstd, label = "Prediction")
plt.plot(BTC_df["midpoint"][train_length + window_size:].reset_index(drop = True), label = "True Price")
plt.legend(loc="upper right")

### LSTM Model

In [ ]:
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(16, input_shape=[30, 154]),
    keras.layers.Dense(1)
    ])
    
model_lstm.compile(loss="mse", optimizer="adam", metrics=["mse"])
history = model_lstm.fit(windows_train, epochs=10,verbose=1)

In [ ]:
# Predict
preds_lstm = model_lstm.predict(windows_test)

# Remove standardization from predictions

preds_lstm_unstd = preds_lstm * std_df["midpoint"] + mean_df["midpoint"]


# Plot 
plt.plot(preds_lstm_unstd, label = "Prediction")
plt.plot(BTC_df["midpoint"][train_length + window_size:].reset_index(drop = True), label = "True Price")
plt.legend(loc="upper right")